In [ ]:
!pip install cobaya

In [ ]:
!cobaya-install cosmo -p /path/to/packages

In [ ]:
# General python imports

import numpy as np
import matplotlib.pyplot as plt
import time
import os, sys

# Import corresponding modules from GetDist
from getdist.mcsamples import loadMCSamples, MCSamplesFromCobaya
import getdist.plots as gdplt


# Matplotlib params set-up for nice plots

%matplotlib inline
plt.rc('xtick',labelsize=16)
plt.rc('ytick',labelsize=16)
plt.rc('font',size=25)
plt.rc('axes', titlesize=26)
plt.rc('axes', labelsize=25)
plt.rc('lines', linewidth=2)
plt.rc('lines', markersize=6)
plt.rc('legend', fontsize=20)

# Advanced tutorial 2


**Description**: this jupyter notebook is designed to help you solve the exercises planned for this advanced practical session. For this, we will use the CMB measurements of `Planck18` to deal as the observational probes to constrain the Standard Cosmological Model. We will use `Cobaya` as the main Bayesian Analysis tool and `CAMB` as the Boltzmann Solver.


### Exercises:

* **Exercise 0**: Plot the chains from the previous tutorial to see the evolution

* **Exercise 1**: Make an evaluation of the posterior distribution by running Cobaya with its model wrapper and the CMB Temperture anisotropies likelihood of Planck18. Keep the cosmological values fixed. Think about the values of the likelihood and the posterior. What are we sampling?

* **Exercise 2**: Plot the CMB Temperature angular power spectra retrieved from `CAMB` via Cobaya given the parameters values fixed in exercise 1.

* **Exercise 3**: Plot the CMB Temperature angular power spectra retrieved from `CAMB` via Cobaya given a very different value for $\Omega_b$. Compare with the result of exercise 2 by making a plot of the difference between the angular power spectra. Calculate the difference in the likelihood. Follow what you have learnt in exercise 1.

* **Bonus exercise**: Repeat everything in exercises 1, 2 and 3 given a modified CAMB that contains a feature in the primordial power spectrum. This exercise will unveil the full power of Cobaya and why it is useful for the Euclid Consortium. There is a dedicated notebook called `Cobaya-BonusExercise.ipynb`.


## Exercise 0:

Plot the chains corresponding to the $\Lambda$CDM model that you left running yesterday to see the evolution.

Alternatively, plot the the chains in the corresponding tar file of the respository.

In [ ]:
# Load the chains and the updated config file from the previous run
gdsamples = loadMCSamples('/LCDM')

gdplot_cartesian = gdplt.get_subplot_plotter(width_inch=5)
gdplot_cartesian.triangle_plot(gdsamples, ['H0', 'ombh2', 'omch2', 'As', 'ns'], filled=True)

## Exercise 1:

Make an evaluation of the posterior distribution by running Cobaya with its model wrapper and the Planck18 TT data set, keeping cosmological values fixed. Think about the values of the likelihood and the posterior. What are we sampling?

In [ ]:
# We are running Cobaya with Planck18 data
# Let's start by defining the corresponding config file
# The options that can be modified by the user are pointed with the acronym (UC).
# Note that currently all the parameters are fixed!

# Fill the info dictionary yourself!

# We are running Cobaya with Planck18 data
# Let's start by defining the corresponding config file
# The options that can be modified by the user are pointed with the acronym (UC).
# Note that currently all the parameters are fixed!

info = {
    #'params': Cobaya's protected key of the input dictionary.
    # Includes the parameters that the user would like to sample over:
'params': {
        # Each parameter below (which is a 'key' of another sub-dictionary) can contain a dictionary
        # with the key 'prior', 'latex'...
        # If the prior dictionary is not passed to a parameter, this parameter is fixed.
        # In this example, we are sampling the parameter ns
        # For more information see: https://cobaya.readthedocs.io/en/latest/example.html
        'ombh2': 0.022445, #Omega density of baryons times the reduced Hubble parameter squared
        'omch2': 0.1205579307, #Omega density of cold dark matter times the reduced Hubble parameter squared
        'H0': 67, #Hubble parameter evaluated today (z=0) in km/s/Mpc
        'tau': 0.0925, #optical depth
        'mnu': 0.06, #  sum of the mass of neutrinos in eV
        'nnu': 3.046, #N_eff of relativistic species
        'As': 2.12605e-9, #Amplitude of the primordial scalar power spectrum
        'ns': 0.965, # primordial power spectrum tilt (sampled with an uniform prior)
        'w': -1, #Dark energy fluid model
        'wa': 0, #Dark energy fluid model
        'omk': 0.0, #curvature density
        'omegam': None, #DERIVED parameter: Omega matter density
        'omegab': None, #DERIVED parameter: Omega baryon density
        'omeganu': None, #DERIVED parameter: Omega neutrino density
        'omnuh2': None, #DERIVED parameter: Omega neutrino density times de reduced Hubble parameter squared
        'omegac': None, #DERIVED parameter: Omega cold dark matter density
        'N_eff': None},
    # 'theory': Cobaya's protected key of the input dictionary.
    # Cobaya needs to ask some minimum theoretical requirements to a Boltzman Solver
    # you can choose between CAMB or CLASS
    # In this notebook, we use CAMB and specify some CAMB arguments
    # such as the number of massive neutrinos
    # and the dark energy model
    #
    'theory': {'camb':
               {'stop_at_error': True,
                'extra_args':{'num_massive_neutrinos': 1,
                              'dark_energy_model': 'ppf'}}},
    #'sampler': Cobaya's protected key of the input dictionary.
    # you can choose the sampler you want to use.
    # Check Cobaya's documentation to see the list of available samplers
    # In this exercise, we use the 'evaluate' sampler to make a single computation of the posterior distributions
    # Note: if you want to run a simple MCMC sampling choose 'mcmc'
    'sampler': {'evaluate': None},
    # 'packages_path': Cobaya's protected key of the input dictionary.
    # This is the variable you need to update
    # if you are running Cobaya with cobaya_modules (option (2) above).
    # If you are using the conda likelihood environment or option (1),
    # please, keep the line below commented
    #
    #'packages_path': modules_path,
    #
    #'output': Cobaya's protected key of the input dictionary.
    # Where are the results going to be stored, in case that the sampler produce output files?
    # For example: chains...
    # (UC): modify the path below within 'output' to choose a name and a directory for those files
    'output': 'chains/euclid_school',
    #'debug': Cobaya's protected key of the input dictionary.
    # how much information you want Cobaya to print? If debug: True, it prints every single detail
    # that is going on internally in Cobaya
    'debug': False,
    #'timing': Cobaya's protected key of the input dictionary.
    # if timing: True, Cobaya returns how much time it took it to make a computation of the posterior
    # and how much time take each of the modules to perform their tasks
    'timing': True,
    #'force': Cobaya's protected key of the input dictionary.
    # if 'force': True, Cobaya forces deleting the previous output files, if found, with the same name
    'force': True,
    }

In [ ]:
# We import the likelihood of Planck18
#'likelihood': Cobaya's protected key of the input dictionary.

info['likelihood'] = {
                'planck_NPIPE_highl_CamSpec.TTTEEE': None}

# Print the full info!
info

In [ ]:
#First: import model wrapper of Cobaya
from cobaya.model import get_model

# The `get_model` wrapper of Cobaya imported in the line above needs a yaml or python dictionary as argument
model = get_model(info)

In [ ]:
# Let's have an insight about what Cobaya has requested to CAMB to calculate the Planck18 likelihood

# (1) Requirements needed by the likelihood code.
# That means, which quantities are we asking to the Boltzman (CAMB/CLASS) through Cobaya?
print('\n Requirements \n')
print(model.provider.requirement_providers)
# (2) At which values have the requirements been requested (redshift, scales...)?
print('\n Requested \n')
print(model.requested())
print('\n Parameters \n')
print(model.input_params)

In [ ]:
# Let's get a random set of sampled parameters values
point = dict(zip(model.parameterization.sampled_params(),
                 model.prior.sample(ignore_external=True)[0]))
print('Sampled parameters values: \n', point)
# Make a computation of the logposterior on a set of parameter values
logposterior = model.logposterior(point)


# Note that we are measuring the time for illustration purposes only.

print('\n Full log-posterior:')
print('   logposterior: %g' % logposterior.logpost)
print('   logpriors: %r' % dict(zip(list(model.prior), logposterior.logpriors)))
print('   loglikelihoods: %r' % dict(zip(list(model.likelihood), logposterior.loglikes)))
print('   derived params: %r' % dict(zip(list(model.parameterization.derived_params()), logposterior.derived)))

## Exercise 2:

Plot the CMB Temperature angular power spectra retrieved from `CAMB` via Cobaya given the values above. Have a look at https://cobaya.readthedocs.io/en/latest/cosmo_external_likelihood.html to get inspiration.

In [ ]:
# Check the lines above to see which requirements were selected by the model when Planck18 was specified
# In particular, look at model.provider


Cls = model.provider.get_Cl(ell_factor=True, units="muK2")
Cls


In [ ]:
# Let's plot them!

plt.plot(Cls['ell'][2:], Cls['ell'][2:] * (Cls['ell'][2:] + 1)*Cls['tt'][2:])
plt.xlabel(r'$\ell$')
plt.ylabel(r'$C_\ell^{TT}$');

## Exercise 3:

Plot again the CMB Temperature angular power spectra retrieved from `CAMB` via Cobaya given a very different value for $\Omega_b$. Keep in mind that you need to fix the nuisance parameters of the Planck18 likelihood. How would you do it?

Make a plot plotting the difference between the corresponding temperature angular power spectra. Calculate the difference in the likelihood value for both cases.

To do this exercise, I recommend you creating a new model instance of the model wrapper.

In [ ]:
# Make a copy of the info dictionary and modify the entry corresponding to Omega_b
info_modified_omegab = info.copy()
info_modified_omegab['params']['ombh2'] = 10 * 0.022445

In [ ]:
# Create a new model instance with this modified info dictionary
model_modified_omegab = get_model(info_modified_omegab)

# Make an evaluation of the posterior so that you can request the requirements\
# Note that you need to do it in the same point as before
logposterior_modified_omegab = model_modified_omegab.logposterior(point)

In [ ]:
# Plot the difference in the Cls
Cls_modified_omegab = model_modified_omegab.provider.get_Cl(ell_factor=False, units="muK2")

# Let's plot them!

plt.plot(Cls_modified_omegab['ell'][2:],
          Cls_modified_omegab['ell'][2:] * (Cls_modified_omegab['ell'][2:] + 1) * (Cls['tt'][2:]-Cls_modified_omegab['tt'][2:])/Cls['tt'][2:])
plt.ylabel(r'$C_\ell^{TT}$');
plt.xlabel(r'$\ell$');


In [ ]:
# Calculate the difference in the likelihood values for both models

logposterior.loglike-logposterior_modified_omegab.loglike